In [80]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import time
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import dask.dataframe as dd

from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LearningCurveDisplay
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error,r2_score
from sklearn.model_selection import GroupKFold, cross_validate, cross_val_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = dd.read_csv('/Users/johnleland/Downloads/FinalFile.csv', header = 0,dtype={'PPFD_OUT': 'float64',
       'SWC_3_4_1': 'float64'})

for (columnName, columnData) in df.items():
    print('Column Name : ', columnName)

Column Name :  YEAR
Column Name :  DateTime
Column Name :  MONTH
Column Name :  DAY
Column Name :  DOY
Column Name :  HOUR
Column Name :  MINUTE
Column Name :  TIMESTAMP
Column Name :  TIMESTAMP_START
Column Name :  TIMESTAMP_END
Column Name :  H2O_1_1_1
Column Name :  H2O_1_1_2
Column Name :  H2O_1_1_3
Column Name :  H2O_1_2_2
Column Name :  H2O_1_2_3
Column Name :  H2O_1_3_2
Column Name :  H2O_1_3_3
Column Name :  H2O_1_4_2
Column Name :  H2O_1_4_3
Column Name :  H
Column Name :  TS_1_1_1
Column Name :  TS_1_2_1
Column Name :  TS_1_3_1
Column Name :  TS_1_4_1
Column Name :  TS_1_5_1
Column Name :  TS_1_6_1
Column Name :  TS_1_7_1
Column Name :  TS_1_8_1
Column Name :  TS_1_9_1
Column Name :  PPFD
Column Name :  PPFD.1
Column Name :  PPFD.2
Column Name :  PPFD.3
Column Name :  PPFD_OUT
Column Name :  PPFD_BC_IN_1_1_1
Column Name :  PPFD_BC_IN_2_1_1
Column Name :  PPFD_BC_IN_3_1_1
Column Name :  Tair
Column Name :  Tair.1
Column Name :  Tair.2
Column Name :  Tair.3
Column Name :  Tair.

In [16]:
#data = df[df.Ustar >= df.U5]
#df["FC5"] = data["FC"]

data = df[df.Ustar >= df.U50]
df["FC50"] = data["FC"]

#data = df[df.Ustar >= df.U95]
#df["FC95"] = data["FC"]

L2 = ['YEAR', 'MONTH', 'DAY', 'DOY', 'HOUR', 'MINUTE', 'TS_1_1_1', 'TS_1_2_1',
       'TS_1_3_1', 'TS_1_4_1', 'TS_1_5_1', 'TS_1_6_1', 'TS_1_7_1', 'TS_1_8_1',
       'TS_1_9_1', 'PPFD', 'PPFD.1', 'PPFD.2', 'PPFD.3', 'PPFD_OUT',
       'PPFD_BC_IN_1_1_1', 'PPFD_BC_IN_2_1_1', 'PPFD_BC_IN_3_1_1', 'Tair',
       'Tair.1', 'Tair.2', 'Tair.3', 'Tair.4', 'VPD_PI', 'NETRAD',
       'Rh', 'HourCos', 'HourSin', 'DOYCos', 'DOYSin', 'VPD', 'Rg',"SWC_1_1_1","SWC_1_2_1","SWC_1_3_1", "Site","FC50"]

L = ["YEAR","MONTH","DAY","DOY","HOUR","MINUTE","FC50","Tair","SWC_1_1_1","SWC_1_2_1","SWC_1_3_1","PPFD","PPFD_OUT","TS_1_1_1","TS_1_2_1","TS_1_3_1",
     "TS_1_4_1","Site","HourCos","VPD","DOYCos","Tair.1","Tair.2"]
df1 = df[L2]
print(df1.columns)

df1 = df1.dropna().compute()

#df1.dropna(inplace=True)

Index(['YEAR', 'MONTH', 'DAY', 'DOY', 'HOUR', 'MINUTE', 'TS_1_1_1', 'TS_1_2_1',
       'TS_1_3_1', 'TS_1_4_1', 'TS_1_5_1', 'TS_1_6_1', 'TS_1_7_1', 'TS_1_8_1',
       'TS_1_9_1', 'PPFD', 'PPFD.1', 'PPFD.2', 'PPFD.3', 'PPFD_OUT',
       'PPFD_BC_IN_1_1_1', 'PPFD_BC_IN_2_1_1', 'PPFD_BC_IN_3_1_1', 'Tair',
       'Tair.1', 'Tair.2', 'Tair.3', 'Tair.4', 'VPD_PI', 'NETRAD', 'Rh',
       'HourCos', 'HourSin', 'DOYCos', 'DOYSin', 'VPD', 'Rg', 'SWC_1_1_1',
       'SWC_1_2_1', 'SWC_1_3_1', 'Site', 'FC50'],
      dtype='object')


In [17]:
# t = list(np.unique(df1["Site"]))
t = df1.Site.unique()
t

array(['US-xDJ', 'US-xDL', 'US-xDS', 'US-xJE', 'US-xKA', 'US-xKZ',
       'US-xMB', 'US-xNQ', 'PR-xLA', 'US-xNW', 'US-xSC', 'US-xSE',
       'US-xSL', 'US-xST', 'US-xTA', 'US-xUN', 'US-xWD', 'US-xAB',
       'US-xBL', 'US-xBR', 'US-xCL', 'US-xDC'], dtype=object)

In [18]:
sub_lists = np.array_split(t, 2)
sub_lists

[array(['US-xDJ', 'US-xDL', 'US-xDS', 'US-xJE', 'US-xKA', 'US-xKZ',
        'US-xMB', 'US-xNQ', 'PR-xLA', 'US-xNW', 'US-xSC'], dtype=object),
 array(['US-xSE', 'US-xSL', 'US-xST', 'US-xTA', 'US-xUN', 'US-xWD',
        'US-xAB', 'US-xBL', 'US-xBR', 'US-xCL', 'US-xDC'], dtype=object)]

model = xgb.XGBRegressor(objective="reg:squarederror", random_state=13)

print("------------------------------Model Created-----------------------------------")

overallmse = []
overallr2 = []
iter = range(1,30)
for i,sitename in zip(iter,t):
    start = time.time()
    print("Beginning iteration:",i)
    ## data setup
    df_test = df1[df1["Site"] == sitename]
    df_train = df1[df1['Site'] != sitename]

    x_train = df_train.drop(["FC50","Site"], axis = 1)
    y_train = df_train['FC50']
    x_test = df_test.drop(["FC50","Site"],axis=1)
    y_test = df_test['FC50']

    #evalset = [(x_train,y_train),(x_test,y_test)]
    # Model
    #model.fit(x_train,y_train, eval_set = evalset, verbose = False)
    model.fit(x_train,y_train, verbose = False)

    #Prediction
    #predict1 = model1.predict(x_test)
    predict2 = model.predict(x_test)

    #Model Assessment
    mse = mean_squared_error(y_test,predict2)
    score = r2_score(y_pred= predict2, y_true=y_test)
    overallmse.append(mse)
    overallr2.append(score)
    end = time.time()
    elapsed = end - start
    print("Time Per Fold:", elapsed)
    print(f'Fold {i}:  MSE: {mse}, R2 Score: {score}')

In [81]:
model = RandomForestRegressor(min_samples_split=2,n_estimators=10,random_state=13, max_depth= 50)

X = df1.drop(["FC50","Site"], axis = 1)
Y = df1["FC50"]

# scores = cross_val_score(model, X, Y, cv=5)

In [83]:
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(X, Y, df1["Site"])

3

In [28]:
print(max(Y), min(Y))

99.7576643792824 -99.6636532993795


In [32]:
validate = cross_validate(model,X,Y, groups = df1["Site"], cv=5)

In [34]:
print(np.mean(validate["test_score"]))
validate["test_score"]

0.29215330975002507


array([ 0.39092296, -0.56613799,  0.6195772 ,  0.59843688,  0.4179675 ])

In [54]:
def outlier(data):
    x = sorted(data)
    q1, q3= np.percentile(data,[25,75])
    iqr = q3 - q1
    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr) 
    return lower_bound,upper_bound

In [56]:
x = outlier(df1["FC50"])
x

(-8.51201645470904, 7.020842240163652)

In [68]:
y = df1[df1.FC50 < 7.02]

y = y[y.FC50 > -8.512]

,YEAR,MONTH,DAY,DOY,HOUR,MINUTE,TS_1_1_1,TS_1_2_1,TS_1_3_1,TS_1_4_1,...,HourSin,DOYCos,DOYSin,VPD,Rg,SWC_1_1_1,SWC_1_2_1,SWC_1_3_1,Site,FC50
34468,2017,8,3,215,10,45,11.340,9.299,6.751,5.739,...,-0.544021,-0.839072,-0.544021,11.434997,1589.40,5.514293,9.225115,7.687249,US-xDJ,-5.257908
34469,2017,8,3,215,11,15,11.609,9.527,6.841,5.743,...,-0.999990,0.004426,-0.999990,11.953441,1848.06,5.513818,9.225955,7.687204,US-xDJ,-7.379187
34470,2017,8,3,215,11,45,11.922,9.748,6.924,5.750,...,-0.999990,0.004426,-0.999990,12.374404,2028.56,5.513342,9.226795,7.687159,US-xDJ,-8.312512
34471,2017,8,3,215,12,15,12.213,9.966,7.009,5.760,...,-0.536573,0.843854,-0.536573,12.743160,1692.34,5.512866,9.227635,7.687114,US-xDJ,-7.704689
34472,2017,8,3,215,12,45,12.415,10.147,7.089,5.772,...,-0.536573,0.843854,-0.536573,12.437413,1637.68,5.512390,9.228475,7.687069,US-xDJ,-5.822808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65178,2022,6,30,181,6,45,16.444,16.780,17.092,16.708,...,-0.279415,0.960170,-0.279415,3.627493,817.50,6.840000,17.330000,23.790000,US-xDC,-3.814404
65179,2022,6,30,181,7,15,16.798,16.979,17.075,16.689,...,0.656987,0.753902,0.656987,3.947518,722.86,6.840000,17.330000,23.790000,US-xDC,-3.856184
65180,2022,6,30,181,7,45,17.079,17.155,17.068,16.672,...,0.656987,0.753902,0.656987,4.651792,1438.48,6.840000,17.330000,23.790000,US-xDC,-4.895776
65199,2022,6,30,181,17,15,25.274,23.773,19.524,17.400,...,-0.961397,-0.275163,-0.961397,12.831164,1891.48,6.840000,17.330000,23.790000,US-xDC,-7.655773


In [82]:
X = y.drop(["FC50","Site"], axis = 1)
Y = y["FC50"]

In [84]:
validate = cross_validate(model,X,Y, groups = y["Site"], cv=group_kfold)

In [79]:
validate

{'fit_time': array([238.58199811, 238.64851809, 241.57434106, 242.84625101,
        235.342623  ]),
 'score_time': array([0.31604004, 0.2899611 , 0.35254812, 0.29900289, 0.36416411]),
 'test_score': array([0.34720499, 0.49183688, 0.2714586 , 0.38153905, 0.38077285])}